## Apiux & SII: calculo de indice de materia oscura en personas juridicas.
## ATENCION: proyecto sujeto a mantenimiento continuo. 

## Henry Vega (henrry.vega@api-ux.com)
## Data analyst

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
#warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:

spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.yarn.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/") \
  .config("spark.executor.memory", "24g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "12") \
  .config("spark.executor.instances", "24") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()

Setting spark.hadoop.yarn.resourcemanager.principal to yasser.nanjari


## Carga de relaciones societarias(depurada)

Se carga la data depurada anteriormente de relaciones societarias.

In [3]:
df = spark.read.options(header=True,inferSchema=True,delimiter=",").csv("../../data/processed/sociedades_participacion_capital_nozero.csv")
df.createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad order by RUT_SOCIEDAD asc").show()

Hive Session ID = a064edd0-387d-4aff-8e0d-1a455af406a2                          
[Stage 2:=================================================>       (33 + 5) / 38]

+--------------------+--------------------+------------------+---------------------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+--------------------+--------------------+------------------+---------------------+
|+++JjoAr1RQa1kY7r...|S2zsiazT64Jiu/sSE...|             85.71|                85.71|
|+++JjoAr1RQa1kY7r...|xO5P1s+Yeug4wKtb8...|             14.29|                14.29|
|+++P6tU1TAUNG0SZv...|8ejqjUOvnKKfMZSzj...|             100.0|                100.0|
|+++UeiptXaAQD74N1...|5jgmhcvBN2sVp7Gh5...|              72.0|                 72.0|
|+++UeiptXaAQD74N1...|QemGi6c6gkD3rdstn...|              28.0|                 28.0|
|+++VKgYcn1igYZjkT...|XRkbvgfaa5MRVblDF...|             100.0|                100.0|
|+++VWXcqX/471v55m...|d/Gh1G45HEItM+byD...|              99.0|                 99.0|
|+++VWXcqX/471v55m...|t9AEh7tBjjp3FKWzm...|               1.0|                  1.0|
|+++ZYfC4XA28yQEQd...|Rf14vs+327ZlNsQvX...|             100.0|   

## Exploracion de composiciones de socios en capital y utilidades

Como no hay unicida de entradas y se puede presentar varias veces una combinacion sociedad y socio, se agrupa por el promedio para cada relacion. De esta forma analizamos la composicion de la sociedad en utilidades y capital.


In [4]:
spark.sql("select  RUT_SOCIEDAD, RUT_SOCIO, mean(PORCENTAJE_CAPITAL) as PORCENTAJE_CAPITAL, mean(PORCENTAJE_UTILIDADES) as PORCENTAJE_UTILIDADES from sociedad group by RUT_SOCIEDAD, RUT_SOCIO").createOrReplaceTempView("composicion")
spark.sql("select  RUT_SOCIEDAD, sum(PORCENTAJE_CAPITAL) as TOTAL_CAPITAL, SUM(PORCENTAJE_UTILIDADES) as TOTAL_UTILIDADES from composicion group by RUT_SOCIEDAD").createOrReplaceTempView("composicion")
m=spark.sql("select * from composicion").toPandas()

Veamos cuantas sociedades suman mas del 100% en capital total.

In [5]:
m[m['TOTAL_CAPITAL']>105] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
35815,qBN0Ew2XbPgKTj0PyLtzcA==,110.00,74.00
42117,0XuN90P2eQMydmKE9OW64Q==,110.00,110.00
43531,+yO9yJXI/gw9x7dec0Lt+A==,105.70,124.99
52356,W62rStOCYMtosUOwv5LBgA==,110.00,110.00
61200,zJLdRqdwJgz4dtWd9xh1Lg==,110.00,110.00
...,...,...,...
1344066,qyJdWYK0NpWG7WDVPw2igQ==,109.00,109.00
1344785,PpKVd7x+KKrDzssyhs0TAw==,106.78,106.78
1353491,p8AtykYwPslwpOXSkRODLQ==,110.00,100.00
1370978,MqkQRluUHZIXHZlneEoJsQ==,109.00,109.00


Veamos cuantas sociedades suman menos de 90% en capital total.

In [6]:
m[m['TOTAL_CAPITAL']<90] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
681134,015/V4NMbTt4NqHquyzHAQ==,11.29,11.29


In [7]:
m[m['TOTAL_UTILIDADES']>105] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
16143,y2vuFSg0i2sqmpg4ayJ4LQ==,100.00,109.00
28423,OdNW1enN1u4LwpLOpweOTQ==,100.00,120.00
38246,3pZ4jwIjTPhgr9SFW72W9g==,100.00,145.00
40087,AFT7+4VOTHnj2AQ4V7HRjA==,100.00,109.00
42117,0XuN90P2eQMydmKE9OW64Q==,110.00,110.00
...,...,...,...
1344785,PpKVd7x+KKrDzssyhs0TAw==,106.78,106.78
1366014,7oq5WRip3ZmPavK3AkWR/A==,100.00,154.00
1370774,vGFtt5wSI+ZAORZgj9Fs/w==,100.00,145.00
1370978,MqkQRluUHZIXHZlneEoJsQ==,109.00,109.00


In [8]:
m[m['TOTAL_UTILIDADES']<95] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
71,kqvmxZXKear6P8xAR3GXVg==,90.00,90.0
382,Q59+BgjpBF+V781RgOgnbQ==,100.00,0.0
708,IRrJwtGlTFSYlh/9OmkSbA==,100.00,93.3
1717,7/JzC/njmhl4xZ3D1Z/lcA==,94.00,94.0
1772,BltJr9zcmXCdMlpgIKgScg==,99.99,75.0
...,...,...,...
1388226,4MdTds72COdPZizHGW5Sxg==,99.99,90.0
1389039,LFSUoUnGK/OcQf/9cZN3cg==,100.00,50.0
1389690,mHXs6IbCeQvL1KVfYly91g==,100.00,50.0
1389934,MGStvbYH0J++QOqIDvrmsw==,100.00,50.0


Se utilizara el porcentaje de capital en lugar de utilidades para hacer la transmision de materia oscura. En el proceso de limpieza de datos se considero las filas con valores no nuels de PORCENTAJE_CAPITAL. Se utilizara una tabla de composicion para poder normalizar los valores de oscuridad para cuando los valores de porcentajes se capital sean diferentes al 100%.

## Tabla de ponderacion para el total de capital.

In [9]:
spark.sql("select RUT_SOCIEDAD as CONT_RUT, TOTAL_CAPITAL as ponderador from composicion").createOrReplaceTempView("composicion")

## Lectura de tabla de oscuridad

Se hace lectura de los datos de iniciales de oscuridad para personas naturales.

In [10]:
oscuridad1=spark.sql("select * from libsdf.jab_materia_inom")
oscuridad1.createOrReplaceTempView("oscuridad")
spark.sql("select * from oscuridad").show()

[Stage 8:======================================================>  (19 + 1) / 20]

+--------------------+------------+
|            CONT_RUT|       Valor|
+--------------------+------------+
|+++++jqhSCxb4MYni...|0.2505881216|
|++++4VUrSBUBG/m/C...|0.2505881216|
|++++LbywbfVQJ4b6r...|         1.0|
|++++mpym73swtBJsE...|0.2505881216|
|+++/mtz2Y6dR0yWCY...|0.2505881216|
|+++0Pn3sll6VwPH5U...|0.2505881216|
|+++0Re2TkLe14DpcD...|0.2505881216|
|+++1gCGSMhlEHLJDQ...|         1.0|
|+++1iAkZoVFs4IYW1...|0.2505881216|
|+++21EA/iuTO4I/ht...|0.2505881216|
|+++2MYr/KzH2JhNCN...|         1.0|
|+++40fb1kG88W0AEf...|         1.0|
|+++4Na3wJ7SIFPhPQ...|         1.0|
|+++58HnadEZ4X4kXn...|         1.0|
|+++5xuy9R938QLB66...|0.7911765654|
|+++69H052vRRRkQ54...|0.2505881216|
|+++6Pf94vKHoUDM+E...|0.2505881216|
|+++7gCpP/p5z+drab...|0.3512938674|
|+++86Ho6cZD74hGWy...|         1.0|
|+++Aa4Gv7OgWPwQ6p...|0.6809641971|
+--------------------+------------+
only showing top 20 rows



## Ajuste de valores de participacion societaria.

Como para una combinacion sociedad socio tenemos distintos valores de PORCENTAJE_CAPITAL y PORCENTAJE_UTILIDADES se usara el promedio de dichos valores.

In [11]:
spark.sql("select  RUT_SOCIEDAD, RUT_SOCIO, mean(PORCENTAJE_CAPITAL) as PORCENTAJE_CAPITAL, mean(PORCENTAJE_UTILIDADES) as PORCENTAJE_UTILIDADES from sociedad group by RUT_SOCIEDAD, RUT_SOCIO").createOrReplaceTempView("sociedad")

## Primera iteracion

Se realiza el cruce de la data societaria con la data de  oscuridad de personas naturales (1re paso de calculo de materia oscura para sociedades completas). Para ello se completa la malla con la oscuridad inicial. Hay sociedades donde no se completan todos los socios con oscuridad, por lo cual se discrimina mediante un contador de nulos de dicho campo para poder agregar la data y obtener un valor de oscuridad para un rut de soiedad donde todas las entradas de sus socios han sido completadas.

In [12]:
#Iteracion 0
spark.sql("select * from sociedad left join oscuridad on sociedad.RUT_SOCIO=oscuridad.CONT_RUT order by sociedad.RUT_SOCIEDAD asc").createOrReplaceTempView("sociedad")
#spark.sql("select * from sociedad ").show()
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, PORCENTAJE_CAPITAL, Valor from sociedad").createOrReplaceTempView("sociedad")
#spark.sql("select * from sociedad ").show()
spark.sql("select RUT_SOCIEDAD as RUT_SOCIEDAD1, count(*) as nulos  from sociedad where Valor is null group by RUT_SOCIEDAD order by RUT_SOCIEDAD ASC").createOrReplaceTempView("aux")
#spark.sql("select * from aux ").show()
spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, nulos, Valor from sociedad left join aux on sociedad.RUT_SOCIEDAD=aux.RUT_SOCIEDAD1 order by RUT_SOCIEDAD asc ").createOrReplaceTempView("aux")
#spark.sql("select * from aux where nulos is null ").show()


## Primera iteracion, cruce con oscuridad de personas naturales (2do paso de calculo de oscuridad y ponderacion por total de capital)

Junto con completar la data con los valores de oscuridad para sociedades completas, se agrega un ponderador. De esta forma, si una entidad tiene dos socios con oscuridad y la composicion suma diferente a 100%, se pondera proporcionalmente para que sumen 100%.

In [13]:

#agregar a la oscuridad la ponderacion adecuada

spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, Valor from aux where nulos is null").createOrReplaceTempView("aux")
#spark.sql("select * from aux").show()
spark.sql("select RUT_SOCIEDAD as CONT_RUT, SUM(PORCENTAJE_CAPITAL*Valor*0.01) as othervalue from aux group by RUT_SOCIEDAD").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad").show()
spark.sql("select CONT_RUT, othervalue as Value from oscuridad").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad").show()

spark.sql("select oscuridad.CONT_RUT as CONT_RUT, Value, ponderador from oscuridad left join composicion on oscuridad.CONT_RUT=composicion.CONT_RUT order by oscuridad.CONT_RUT desc ").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad ").show()
spark.sql("select CONT_RUT, Value/ponderador*100 as Value from oscuridad ").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad order by Value desc ").show()

#spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_UTILIDADES, Valor from aux where nulos is null").createOrReplaceTempView("aux")

oscuridad_aux=spark.sql("select * from oscuridad ").toPandas()
oscuridad_aux['iterations']=0


In [14]:
oscuridad_aux

,CONT_RUT,Value,iterations
0,zzzo7RUe8Wp/sBVZrqC1BQ==,0.235367,0
1,zzzEfsBUrus3+LpvpwcHig==,0.250588,0
2,zzz9RqdyHZm0pUPELnEPhw==,0.250588,0
3,zzz0aRgQrVHwDX+AyZc8SQ==,0.250588,0
4,zzydn/q5KUdW0PcyxWSyzg==,0.250588,0
...,...,...,...
1284452,+++VWXcqX/471v55mS2YiA==,0.250588,0
1284453,+++VKgYcn1igYZjkTHXjgA==,0.250588,0
1284454,+++UeiptXaAQD74N1rCuIg==,0.250588,0
1284455,+++P6tU1TAUNG0SZvJeW6w==,0.250588,0


In [15]:
#soc = spark.sql("SELECT RUT_SOCIEDAD, PORCENTAJE_CAPITAL, Valor FROM sociedad WHERE count(RUT_SOCIO) = 1 GROU").toPandas()
#soc = spark.sql("SELECT * FROM sociedad").toPandas()
soc = spark.sql("SELECT RUT_SOCIEDAD AS CONT_RUT, COUNT(RUT_SOCIO) AS N_SOCIOS FROM sociedad GROUP BY RUT_SOCIEDAD").toPandas()

In [16]:
soc2 = soc[soc["N_SOCIOS"]==1]
soc2

,CONT_RUT,N_SOCIOS
1,+++P6tU1TAUNG0SZvJeW6w==,1
3,+++VKgYcn1igYZjkTHXjgA==,1
5,+++ZYfC4XA28yQEQdCy5yw==,1
10,++/UTl2mwL/J484yKcuZrg==,1
16,++0hvpY1Z3Wx9DKo2e+d8Q==,1
...,...,...
1391755,zuGtDs1T0+D7MOHk+pfbRA==,1
1391757,zvzzFABjkreNPKE5BoZoyw==,1
1391758,zxH9rTmf0UhJVhKDsXm3Lg==,1
1391759,zxvYhIbIyLna/W4eEuPlBg==,1


In [17]:
import pandas as pd
aux = pd.merge(soc2,oscuridad_aux, on = ["CONT_RUT"])
aux = aux.rename(columns = {"CONT_RUT": "RUT_SOCIEDAD"})
aux

,RUT_SOCIEDAD,N_SOCIOS,Value,iterations
0,+++P6tU1TAUNG0SZvJeW6w==,1,0.250588,0
1,+++VKgYcn1igYZjkTHXjgA==,1,0.250588,0
2,+++ZYfC4XA28yQEQdCy5yw==,1,0.250588,0
3,++/UTl2mwL/J484yKcuZrg==,1,0.250588,0
4,++0hvpY1Z3Wx9DKo2e+d8Q==,1,0.250588,0
...,...,...,...,...
668481,zuGtDs1T0+D7MOHk+pfbRA==,1,0.250588,0
668482,zvzzFABjkreNPKE5BoZoyw==,1,0.250588,0
668483,zxH9rTmf0UhJVhKDsXm3Lg==,1,0.250588,0
668484,zxvYhIbIyLna/W4eEuPlBg==,1,0.351294,0


In [18]:
dataframe = oscuridad1.toPandas().rename(columns = {"CONT_RUT": "RUT_SOCIO"})
dataframe

,RUT_SOCIO,Valor
0,+++++jqhSCxb4MYniQAmHA==,0.250588
1,++++4VUrSBUBG/m/Cc0FNg==,0.250588
2,++++LbywbfVQJ4b6r4HjYQ==,1.000000
3,++++mpym73swtBJsE7+7Cg==,0.250588
4,+++/mtz2Y6dR0yWCYFksrQ==,0.250588
...,...,...
26523200,HmZli575MBm17Fgi4NsWiw==,1.000000
26523201,HmZlrkb5jYnsZdr+1SPq5Q==,0.250588
26523202,HmZlyXxRicNLaEBZ/Azo3w==,0.250588
26523203,HmZm4Xy1F9NaS8Dv3yHAew==,1.000000


In [19]:
df1 = df.toPandas()
df1

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES
0,dwElNqcCQQiFyI3icvEMrA==,dwElNqcCQQiFyI3icvEMrA==,100.00,100.00
1,E81vZtfA4Ybc4DMafqcW+g==,3BC/zRqDU+1GjN2KsR/Wxg==,99.00,99.00
2,tC5mxZiKq7cNSWyNF3Oz2Q==,C4poSpbJFI6kSghEqd+Qfg==,0.01,0.01
3,E81vZtfA4Ybc4DMafqcW+g==,8pRJYqtj522N9P1N80998Q==,1.00,1.00
4,2AlrCTNxh4TadQyxCtt94w==,d4QdUJK8I1t8jAjfbmuF2g==,70.00,70.00
...,...,...,...,...
2568791,UiRj63CHSbtgGr2EIuyKOg==,TyUdR3KdUravR/JkFTQvdw==,80.65,80.65
2568792,oog+PWTX5phRHz6pHcb2pA==,XUSNfKy4pya9V9PAPov7AQ==,25.00,25.00
2568793,YbAgdHVq63NWCgxJe88K/Q==,4pnL2kHraFroTgfyuJjkEA==,50.00,50.00
2568794,SnaY3i824958Ykq24d23/Q==,hSBNQ4SiQCIyTdgKtM2xSA==,99.80,99.80


In [20]:
aux2 = pd.merge(pd.merge(df1, aux, on = ["RUT_SOCIEDAD"]), dataframe, on = ["RUT_SOCIO"])
aux2 = aux2.rename(columns = {"Value":"Oscuridad_PJ","Valor":"Oscuridad_PN"})
aux2 = aux2.sort_values(by=["Oscuridad_PN"])
aux2

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES,N_SOCIOS,Oscuridad_PJ,iterations,Oscuridad_PN
534142,akLBJ8qb3BhH9PMtcAPQxw==,E4BvIUhzqD3tmlWb157WQA==,100.0,100.0,1,0.0,0,0.0
407855,5wkcRhVqQO78a14jRjwSmw==,RD5agAiDewQpqGq4UupJig==,100.0,100.0,1,0.0,0,0.0
23556,KsUQwhy6wZuyisa2nyKkCw==,LMj/EC/d8y+b2K/VvN0zgw==,100.0,100.0,1,0.0,0,0.0
263923,+Uf1NBGcwO3m+dakWer2jQ==,G4YgRzlqnAmfyuas3e0KBw==,100.0,100.0,1,0.0,0,0.0
64525,+6emXda4RD209rx9QITTMQ==,ZUVMaIekctCUk36jUg+Eag==,100.0,100.0,1,0.0,0,0.0
...,...,...,...,...,...,...,...,...
602809,iRTLuiiWxm+bvD1iUualWg==,sFHGG+CBQal1YZNSoDIHTA==,100.0,100.0,1,1.0,0,1.0
459448,9lzqDgsNzrjmgZchj1VAjQ==,VACJp6d6gXPeh3eGoJCYHg==,100.0,100.0,1,1.0,0,1.0
147026,sT5huY/6Tg0sL21qIjKNuQ==,Bayo5BDmc6QJJtmAgIaMAg==,100.0,100.0,1,1.0,0,1.0
233172,aYSxvOr/i4C5p4Zqhe+Wbw==,pEmVQRh4+IrMDpjyl5VwPg==,100.0,100.0,1,1.0,0,1.0


In [21]:
#aux2[aux2["RUT_SOCIEDAD"]== "XYtAg9XPdlsmKvL99k3t/w=="].head()
aux2[aux2["RUT_SOCIO"]== "1iRqdQ1rnQ51MzfoymrVdQ=="].head()

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES,N_SOCIOS,Oscuridad_PJ,iterations,Oscuridad_PN
45948,QoBskk0DKAc9372RZeJ2NQ==,1iRqdQ1rnQ51MzfoymrVdQ==,100.0,99.99,1,0.250588,0,0.250588
45949,Z43l8Mb6AMvmvEXllZuLqQ==,1iRqdQ1rnQ51MzfoymrVdQ==,100.0,100.00,1,0.250588,0,0.250588
45950,n13iFTWSlPNpDP9cTQgFMA==,1iRqdQ1rnQ51MzfoymrVdQ==,100.0,100.00,1,0.250588,0,0.250588
45951,quaOVnHDb7jYXVDfQstn2g==,1iRqdQ1rnQ51MzfoymrVdQ==,100.0,100.00,1,0.250588,0,0.250588
45952,2C35GQypvxMtgZPfoXlezQ==,1iRqdQ1rnQ51MzfoymrVdQ==,100.0,100.00,1,0.250588,0,0.250588


In [23]:
import numpy as np
aux2["Comp"] = np.where( aux2["Oscuridad_PJ"] == aux2["Oscuridad_PN"],1,0)

In [25]:
aux2[aux2["Comp"]==0]

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES,N_SOCIOS,Oscuridad_PJ,iterations,Oscuridad_PN,Comp
53144,ktucjdqJTykYtmFs5OQhag==,+hR2b8SVLEjixKYww3JmqA==,100.0,100.0,1,0.100706,0,0.100706,0
541339,lYN2TOTYjScSzj5o+3kV9Q==,9QYPv3uVseXjHTKVkpU9GQ==,100.0,100.0,1,0.100706,0,0.100706,0
629789,p2xrw9h1ariK9bnQm3LQNw==,dZHrZe/nB4ui1KSZWh9meg==,100.0,100.0,1,0.100706,0,0.100706,0
260258,wb9CRaFHH7jdgPzAYC/TSA==,afNPVbxJ0vrka5ZRrl90kA==,100.0,100.0,1,0.100706,0,0.100706,0
170587,LqWLlx9eu+wnOUSxRQlDNA==,8swXcsslv0+sOhF5uhJA1A==,100.0,100.0,1,0.100706,0,0.100706,0
...,...,...,...,...,...,...,...,...,...
408075,fBiJxziTKqu7WHj2jJFlQQ==,I+hScMCkKM9lIm9APhSeFA==,100.0,100.0,1,0.791177,0,0.791177,0
601099,VUyRj9xU0RxLxeuIdSt4ZA==,wcPtDhT8PfZ2956RFFgzrQ==,100.0,100.0,1,0.791177,0,0.791177,0
144985,3ocyIYFkEO9qjoAGaLpvog==,nkLgTCmM7q6JegdT9NyxNw==,100.0,100.0,1,0.879153,0,0.879153,0
324182,HEtKjhaYxkRwp7fPKCKVXQ==,14vLum2RljWVNGIzTCPS6w==,100.0,100.0,1,0.879153,0,0.879153,0


## Primera iteracion,resultados

Principales resultados de la primera iteracion, es decir la iteracion cero. 

In [ ]:
oscuridad_aux.describe()

## Iteraciones subsiguientes

Se repite el proceso iterativo para asi completar mas sociedades. El parametro *iter* hara referencia al numero de iteraciones luego de la iteracion 0.

In [ ]:
iter=6

In [ ]:
for iteration in range(1,iter):
    
    spark.sql("select * from sociedad left join oscuridad on sociedad.RUT_SOCIO=oscuridad.CONT_RUT order by sociedad.RUT_SOCIEDAD asc").createOrReplaceTempView("sociedad")
    spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, PORCENTAJE_CAPITAL, CASE WHEN Valor is null THEN Value ELSE Valor END AS Valor from sociedad").createOrReplaceTempView("sociedad")
    spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, PORCENTAJE_CAPITAL, Valor from sociedad").createOrReplaceTempView("sociedad")
    spark.sql("select RUT_SOCIEDAD as RUT_SOCIEDAD1, count(*) as nulos from sociedad where Valor is null group by RUT_SOCIEDAD order by RUT_SOCIEDAD ASC").createOrReplaceTempView("aux")
    spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, nulos, Valor from sociedad left join aux on sociedad.RUT_SOCIEDAD=aux.RUT_SOCIEDAD1 order by RUT_SOCIEDAD asc ").createOrReplaceTempView("aux")
    spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, Valor from aux where nulos is null").createOrReplaceTempView("aux")
    spark.sql("select RUT_SOCIEDAD as CONT_RUT, SUM(PORCENTAJE_CAPITAL*Valor*0.01) as othervalue from aux group by RUT_SOCIEDAD").createOrReplaceTempView("oscuridad")
    spark.sql("select CONT_RUT, othervalue as Value from oscuridad").createOrReplaceTempView("oscuridad")
    
    spark.sql("select oscuridad.CONT_RUT as CONT_RUT, Value,ponderador from oscuridad left join composicion on oscuridad.CONT_RUT=composicion.CONT_RUT order by oscuridad.CONT_RUT asc ").createOrReplaceTempView("oscuridad")
    #spark.sql("select * from oscuridad ").show()
    spark.sql("select CONT_RUT, Value/ponderador*100 as Value from oscuridad order by Value desc ").createOrReplaceTempView("oscuridad")
    #spark.sql("select * from oscuridad ").show()    
    oscuridad=spark.sql("select * from oscuridad ").toPandas()
    oscuridad['iterations']=iteration

    oscuridad=oscuridad.merge(oscuridad_aux, on = "CONT_RUT", how = "left")
    oscuridad['iterations']=oscuridad[["iterations_x", "iterations_y"]].min(axis=1)
    oscuridad = oscuridad.rename(columns={'Value_x': 'Value'})
    oscuridad=oscuridad[['CONT_RUT','Value','iterations']]
    oscuridad_aux=oscuridad.iloc[:,:]
    print(oscuridad_aux.describe())

## Resultados y guardado en archivo csv

Luego de estas iteraciones, se obtienen dos outputs. En primer lugar una tabla actualizada de sociedades y socios con cada valor calculado (o no) de la oscuridad de sus socios. 
En segundo lugar la oscuridad calculada para sociedades.

In [ ]:
sociedad=spark.sql("select * from sociedad").toPandas()

In [ ]:
# Se guardan los archivos finales.
sociedad.to_csv('artefactos/Oscuridad/Sociedades_oscuridad_actualizada.csv', index=False)
oscuridad_aux.to_csv('artefactos/Oscuridad/Sociedades_oscuridad_completa.csv', index=False)
